# Dimensionality Reduction

In this lab, we will work with an IMDB dataset to estimate the sentiment of movie reviews. We will study PCA and Sparse PCA in this context, and work using Single Value Decomposition to perform topic analysis. In the context of text mining, we call SVD *Latent Semantic Analysis* (LSA).

LSA is already implemented in Python in scikit-learn in the package [*TruncatedSVD*](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.TruncatedSVD.html), we will use that along with the Natural Language Processing library [*NLTK*](https://www.nltk.org/) for our methods.

The general process can be summarized as follows:

1. Load the text in free form.
2. Preprocess the text to normalize it.
3. Calculate LSA.
4. Explore the results.

## Loading text: IMDB database.

This dataset comes from the website Internet Movie Database, and represents 25,000 reviews which were labeled (by humans) as positive or negative, see [here](http://ai.stanford.edu/~amaas/data/sentiment/) for more details. It is a pretty big dataset, so we will work with small samples of 500 positive cases and 500 negative cases.

The uncompressed data is simply a series of text documents, each in its own text file, stored in two classes (Positive and Negative), one per folder:

The first step is to load the data and create a "corpus". A corpus is, quite simply, a set of documents. Here, we will read the files from our folders, and assign it a sentiment. We need to read the documents one by one, and store them into a data object which will have the texts and a tag highlighting whether they are positive or negative.

### Reading the text

The first step is to read the data into a vector. We need to read from the document path, using the internal system. This package is called `os` and comes pre-installed in Python.


In [ ]:
# imports
import os
import numpy as np; seed = 2316; np.random.seed(seed)
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
from   scipy.sparse import csr_matrix
import sklearn.feature_extraction.text as sktext
from   sklearn.decomposition import PCA, SparsePCA, TruncatedSVD

# https://scikit-learn.org/stable/modules/manifold.html
from   sklearn.manifold import TSNE



import umap.plot # good for plotting gigantic data

In [ ]:
import umap.umap_ as umap

In [ ]:
# List all files in the positive samples. Replace with your own!
dir = 'C:/Users/jojojoe/Documents/DS 3000_9000 TA/Week_11/Week_11/LSA_Sample/Lecture_Sample/train/pos/'
fileList = os.listdir(dir)

# Create vector with texts
outtexts = []

# Read the files in the directory and append them with the class to the dataset
for eachFile in fileList:
    with open(dir + eachFile, 'rb', newline = None) as _fp:
        fileData = _fp.read()
        outtexts.append(fileData)
    _fp.close()

# Create dataframe from outputs
texts = pd.DataFrame({'texts': outtexts, 'class': 1})
print(texts.shape)
texts.head()

In [ ]:
# Repeat for negative values
# List all files in the "pos" directory
dir = 'C:/Users/jojojoe/Documents/DS 3000_9000 TA/Week_11/Week_11/LSA_Sample/Lecture_Sample/train/neg/'
fileList = os.listdir(dir)

# Create vector with texts
outtexts = []

# Read the files in the directory and append them with the class to the dataset
for eachFile in fileList:
    with open(dir + eachFile, 'rb', newline = None) as _fp:
        fileData = _fp.read()
        outtexts.append(fileData)
    _fp.close()

# Create dataframe from outputs
texts = pd.concat((texts, pd.DataFrame({'texts': outtexts, 'class': 0})), ignore_index = True)
texts.tail()

In [ ]:
texts.describe()

In [ ]:
##See how the text looks like

The text is quite dirty, so we'll use regex code to clean it. It is available in Python using the package [re](https://www.rexegg.com/regex-quickstart.html). Regex can be daunting, but it is very rewarding to learn. Do spend some time with it!

In [ ]:
CLEANR = re.compile('<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});')
def cleanhtml(raw_html):
    html = raw_html.decode('ISO-8859-1') # Change the encoding to your locale!
    cleantext = re.sub(CLEANR, '', html)
    return cleantext

texts['texts'] = texts['texts'].apply(cleanhtml)
texts

Now we will transform the text. The following code uses sklearn's [TfidfVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html) which applies a [Term Frequency - Inverse Document Frequency](https://en.wikipedia.org/wiki/Tf%E2%80%93idf) transformation to the text ( which means counting how many times a certain concept, *e.g.*, a word, appears in the document versus the total times it appears in the document) to do the following:

1. Eliminate accents and other characters.
2. Eliminate the so-called "stopwords", or words that are irrelevant to the learning given they are only connectors. These words are [here](https://gist.github.com/ethen8181/d57e762f81aa643744c2ffba5688d33a).
3. Eliminate concepts that are rare (min_df) or too common (max_df). Here we eliminate concepts that appear in less than 5% of documents and those that appear in over 90%.

The last argument calculates a logaritmic (or sublinear) transformation, which is more robust. This effectively transforms our text data into a fully numeric dataset!


In [ ]:
# Transform the text
TfIDFTransformer = sktext.TfidfVectorizer(strip_accents=, # Eliminate accents and special characters
                      stop_words=, # Eliminates stop words.
                      , # Eliminate words that do not appear in more than 5% of texts
                      , # Eliminate words that appear in more than 95% of texts
                      # Use sublinear weights (softplus), i.e., replace tf with 1 + log(tf)
                      )

The model structure of scikit-learn follows always the same:

1. We define the model using the appropriate function directly from the package (as above).

2. We train the model using the "fit" method over the object we created in 1.

3. We apply the model to data using the "transform" method.

In cases where we want to fit *and* transform the inputs - such as a TF-IDF transform, which is applied over the same data where the weights are "trained" - we can use directly the method "fit_transform", that performs steps 2 and 3 directly.

In [ ]:
TfIDF_IMDB = TfIDFTransformer.
TfIDF_IMDB

It's sparse because not every word appears in every document, leading to null entries. These matrices only store the relevant information! They are much more efficient in memory use and compute time. Unlike operations with dense matrices, operations with sparse matrices do not perform unnecessary low-level arithmetic, such as zero-adds (x+0 is always x). The resulting efficiencies can lead to dramatic improvements in execution time for programs working with large amounts of sparse data. For a full matrix, you store 8 bytes (one double) per entry. For a sparse matrix, you store 12 bytes per entry (one double for the value, and one integer for the column index of the entry).

Here, we have $23848$ sparse entries: $23848 \times 12=286176$ bytes, and in the case of dense we have $1000 \times 230 \times 8 = 1840000 $ bytes. Since $ 286176 < 1840000 $, using sparse should yield better computational efficiency. Use sparse as long as $ 23848 \times 12 < 1000 \times 230 \times 8 $ (or, $ 23848<67\% (1000 \times 230)$).

 `TfIdfVectorizer.fit_transform()` has a `toarray()` method if needed.

FYI: If you have a matrix with a lot of null entries, you could convert it to a sparse matrix using `from scipy.sparse import csr_matrix`.

In [ ]:
data_size = # or 1000*230*8/(1024**2)
print('Size of dense matrix: '+ '%3.2f' %data_size + ' MB')

data_csr_size=
print('Size of sparse csr_matrix: '+ '%3.2f' %data_csr_size + ' MB') # or 23848*12/(1024**2)

In [ ]:
df1 = pd.DataFrame(TfIDF_IMDB.toarray(), columns=TfIDFTransformer.get_feature_names_out())
df1.head()

In [ ]:
vectorizer = sktext.CountVectorizer(, # Eliminate words that appear in more than 95% of texts
                      )
matrix =
df2 = pd.DataFrame(matrix.toarray(), columns=vectorizer.get_feature_names_out())
print(df2.shape)
df2.head()

In [ ]:
# Show us the top 10 most common words
df2
# df2.sum().sum()

The output of the TF-IDF transformer is a sparse matrix. We can check the outputs of the first row with the below code.

In [ ]:
print(TfIDF_IMDB[0,:])

In [ ]:
# and we can verify:
df1.iloc[0,173]

 The following vector shows the list of words associated to each index for indices 30 to 39.

In [ ]:
# Let's save the indexes for later.
word_index =
print(word_index[30:40])

## [PCA](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html) and [Sparse PCA](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.SparsePCA.html)

You have an $n\times p$ data matrix $\boldsymbol X$, where $n$ is the number of samples and $p$ is the number of features. The singular value decomposition (SVD) of $\boldsymbol X$ gives three matrices $\boldsymbol{U\Sigma V}^*$. Combining the first two ($\boldsymbol {Z=U\Sigma}$) gives the matrix of principal components. $\boldsymbol {\Sigma }$ is a diagonal $p\times p$ matrix containing the singular values in descending order. $\boldsymbol {U}$ and $\boldsymbol {V}^*$ are unitary/orthogonal matrices (*i.e.*, rotation matrices).

$\boldsymbol V$ contains the principal loading vectors. This means that the principal components are derived by using the principal loadings as coefficients in a linear combination of your data matrix.

Once you have all the PCs (*i.e.*, $\boldsymbol {U\Sigma }$) figured out you can use the eigenvalues (the sum of squares of the distances from the origin after the projection of the datapoints to the PC, in other words, the square of singular values) to determine the proportion of variation that each PC accounts for.

In [ ]:
# Do normal PCA on the data set
n = 2
nPCA =

# Now we fit. We need to transform our matrix to dense format first.
nPCA.fit(TfIDF_IMDB.toarray())

# Let's calculate the variance of the two components. (lambda_i over sum of lambdas gives amount of explained variance)
total_variance=
print('Total variance explained by the first %i components is %.3f.' % (nPCA.n_components_, total_variance))

total_variance_ratio =
print('The first %i components explain %.3f%% of total variance.' % (nPCA.n_components_, total_variance_ratio))

# Let's get the components and plot them, coloring by the class
Z1 =
sns.scatterplot(x=Z1[:, 0], y=Z1[:, 1], hue=texts['class'])

plt.xlabel("Component 1")
plt.ylabel("Component 2")
plt.title("PCA of IMDB texts")
plt.show()

In [ ]:
### SVD with numpy

X = TfIDF_IMDB.toarray()
X = X - X.mean(axis=0)
print("Shape of X:", X.shape)

u, s, vt = np.linalg.svd(X, full_matrices=False, compute_uv=True) # It's not necessary to compute the full matrix of U or V

# flip eigenvectors' sign to enforce deterministic output
def svd_flip(u, v):
        max_abs_cols = np.argmax(np.abs(u), axis=0)
        signs = np.sign(u[max_abs_cols, range(u.shape[1])])
        u *= signs
        v *= signs[:, np.newaxis]
        return u, v

u, vt = svd_flip(u, vt)

# The columns of u are the principal components scaled to unit norm.
print("\nShape of U:", u.shape)

print("Shape of S:", s.shape)

# The columns of v contain the principal axes.
print("Shape of V.T:", vt.shape)

print("\nCan X be reconstructed using U, S, and V.T?\n{}".format(np.allclose(X, np.dot(u * s, vt))))

print("\nFirst {} singular values calculated manually:\n{}".format(n, s[:n]))

# get the singular values from sklearn
print("\nFirst {} singular values calculated by sklearn:\n{}".format(n, nPCA.singular_values_))

To a obtain a reduced dataset $\boldsymbol X_{d}$, compute the matrix multiplication of the training set matrix $\boldsymbol X$ by the matrix containing the first $d$ eigenvectors (columns) of $\boldsymbol V$.

$\boldsymbol V$: Principal axes in feature space, representing the directions of maximum variance in the data.

In [ ]:
print(X.shape)

d=2
(X@vt.T[:,:d]).shape # this gives a reduced dataset which is 2 dimensional

In [ ]:
print(vt.shape)
vt[0,:5]

In [ ]:
print(nPCA.components_.shape) # of shape (n_components, n_features)
nPCA.components_[0,:5]

In [ ]:
loadings = pd.DataFrame(nPCA.components_.T, columns=['PC1', 'PC2'], index=word_index)
loadings
# Principal axes in feature space, representing the directions of maximum variance in the data.
# Equivalently, the right singular vectors of the centered input data, parallel to its eigenvectors.

# The columns of the dataframe contain the eigenvectors associated
# with the first two principal components. Each element represents
# a loading, namely how much (the weight) each original variable
# contributes to the corresponding principal component.

In [ ]:
# Get explained variance from class attribute:
# The variance of the training samples transformed by a projection to each component (i.e. the eigenvalues of the covariance matrix)
nPCA.explained_variance_ # contains the diagonal elements of the covariance of the two principal components

In [ ]:
# Calculate explained variance manually (from the covariance matrix):
explained_variance=np.cov(Z1.T).diagonal()
explained_variance

In [ ]:
# Get percentage of variance explained by each of the selected components from class attribute:
print("Explained variance ratio:", nPCA.explained_variance_ratio_)

print("Sum of explained variance ratio:", sum(nPCA.explained_variance_ratio_))
# The sum does not add up to 1 implying that the rest of
# the variances are contained in the other components.

Explained Variance Ratio = $\frac{Explained Variance}{\sum Explained Variance}$

To hard code the above formula, must make sure to include in the denominator all variances and not only the variance explained by the components that you have shrunken into. And for that, need to calculate the trace of the covariance of the original X (i.e. features matrix) before PCA.

In [ ]:
# Calculate percentage of variance explained by each of the selected components manually:

# total_variance=np.sum(np.cov(TfIDF_IMDB.toarray().T).diagonal())
# or
total_variance=np.sum(np.trace(np.cov(TfIDF_IMDB.toarray().T)))

explained_variance_ratio=explained_variance/total_variance

print("Sum of explained variance ratio:", sum(explained_variance_ratio))
# The sum does not add up to 1 implying that the rest of
# the variances are contained in the other components.

In [ ]:
nPCA.components_.shape

In [ ]:
len(word_index[nPCA.components_[0,:] != 0]) # all 230 features showed up on the 1st PC

In [ ]:
# Words that are positively correlated to the first component
word_index[nPCA.components_[0,:] > 0] # < 0 gives words with negative correlation

In [ ]:
# Words that are positively correlated to the second component
word_index[nPCA.components_[1,:] > 0]

Even though we only explain 3.4% of the variance in the data we still see some separation!

Regular PCA has a major shortcoming: each principal component is a linear combination of all the original features, and their coefficients are typically non-zero. This can make interpretation hard, especially when a certain number of principal components is chosen.

Sparse PCA finds the set of sparse components that can optimally reconstruct the data. The amount of sparseness is controllable by the coefficient of the L1 penalty, given by the parameter `alpha`.

Sparsity is enforced most of the time on the principal loadings, _i.e.,_ entries of $\boldsymbol V$. The advantage of sparsity is that a sparse $\boldsymbol V$ tells us which variables from the original $p$-dimensional feature space are worth keeping. This helps with interpretability.

Spareness can also be imposed on $\boldsymbol {Z}$. This will make each observation loading appear on
few principal vectors. This is less popular.

Let's see if [Sparse PCA](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.SparsePCA.html) does it better:

In [ ]:
# Now do sparse PCA enforcing sparseness on variables
# that means only a few of the original variables can appear on each latent factor
sPCA =  # play with alpha to control sparsity. Higher values lead to sparser components.
sPCA.fit(TfIDF_IMDB.toarray())

# Get the results
Z2 =

# Create plot
sns.scatterplot(x=Z2[:, 0], y=Z2[:, 1], hue=texts['class'])
plt.xlabel("Component 1")
plt.ylabel("Component 2")
plt.title("Sparse PCA of IMDB texts")
plt.show()

What happened? Given that we are forcing only some of the 230 components to be a part of the solution, this actually hurts the ability to get the components. Still, it allows to study if some components are more relevant! Now you would check which ones are actually present, like so.

In [ ]:
print(sPCA.components_.shape)
len(word_index[sPCA.components_[0,:] != 0]) # only 60 features (out of 230) showed up on the 1st PC

In [ ]:
# Words that are positively correlated to the first component
word_index[sPCA.components_[0,:] > 0]

In [ ]:
# Words that are positively correlated to the second component
word_index[sPCA.components_[1,:] > 0]

As we can see, the first two components are simply referring to the same thing. People that discusses movies and films. Go back and play around with the parameters and see whether you can improve these results.

To get meaningful answers, we need to use truncated Singular Value Decomposition.

## Running [TruncatedSVD](http://scikit-learn.org/stable/modules/generated/sklearn.decomposition.TruncatedSVD.html) Decomposition


This estimator does not center the data before computing the singular value decomposition. This means it can work with sparse matrices efficiently. In particular, truncated SVD works effectively on term count/tf-idf matrices. In that context, it is known as latent semantic analysis (LSA).
Its advantage over PCA is that it works out of the box on sparse data.


Again, we first define it using the TruncatedSVD function.

In [ ]:
svd = TruncatedSVD( # Random state. As SVD is rotation-invariant, we need to set this
                  )

As we will apply the model to new data too, we need to first fit the model, not fit and transform simultaneously.

In [ ]:
# Train the model!
svd.fit(TfIDF_IMDB)

Now we can dig deeper into the model outputs. First, we calculate the explained variance.

In [ ]:
# Get total explained variance in percentage
print('The components explain %.1f%% of total variance.' % (svd.explained_variance_ratio_.sum()*100))
# svd.explained_variance_ratio_

The explained variance is approximately $68\%$ of the total variance using 100 components.

Let's study the relationship between components and words. The matrix *components*, inside our svd object, contains the **principal component matrix**.

In [ ]:
# Get component x words matrix
svd.components_

In [ ]:
svd.components_.shape

We can focus on particular words or concepts too. For example, for the word "action" (in place 3) we can get the following weight vector.

In [ ]:
word_index[3]

In [ ]:
svd.components_[:, 3]

This means that the word "action" is positively related to concept 1, barely negatively related to concept 2 (i.e. that concept does *not* relate to "action"), etc.

To get the five words that relate the most with concept 2, we need to reorder and sort the vector. This can be tricky, so we will use Numpy's *argpartition* function, which will give us the **unsorted** top X values. See the discussion [here](https://stackoverflow.com/questions/6910641/how-to-get-indices-of-n-maximum-values-in-a-numpy-array).

In [ ]:
# To get the second concept, remember that Python starts indexing from 0.
indices = np.argpartition(svd.components_[1, :], -10)[-10:]
indices

In [ ]:
# Get the words.
word_index[indices]

As we can see, the second concept appears to relate to "bad movies". Try concept 1; it will relate to "popular movies".

The code below displays the singular values.

In [ ]:
# Get singular values
svd.singular_values_

Now we have a much better way to study these concepts! Of course, now we can study particular concepts or particular words, as desired.

More importantly, we can now train a model over our reduced space, by using directly the components data matrix as our dataset!

## Non-Linear Visualization

Can we improve this analysis using non-linear methods? We will now study the use of t-Distributed Stochastic Neighbor Embedding (t-SNE) and Uniform Manifold Approximation and Projection (UMAP) over the data. We will use two implementations:

1. t-SNE is available in sklearn, in the [sklearn.manifold](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.manifold) subpackage.
2. The more efficient UMAP is available in its own package, called [umap](https://umap-learn.readthedocs.io/en/latest/basic_usage.html).

### t-SNE

In [ ]:
tSNEmapper = TSNE(n_components=2,               # How many dimensions to use. Never more than 2 or 3
                  init='random',                # First initialization. Sparse matrices need 'random'.  Otherwise use 'pca'
                  perplexity=50.0,              # Read below
                  early_exaggeration=12.0,      # Read below
                  learning_rate='auto',         # Related to above. Leave to auto
                  n_iter=5000,                  # Very important to let iterate enough
                  n_iter_without_progress=300,  # Set early stopping
                  metric='euclidean',           # Metric to use to calculate distances.
                  min_grad_norm=1e-7,           # Minimum gradient to continue iterating
                  verbose=0,                    # Verbosity
                  random_state=seed,            # Random seed
                  n_jobs=-1,                    # Parallel processes
                 )

Some parameters in TSNE are extremely important. [This](https://distill.pub/2016/misread-tsne/) is a great paper going into detail in this regard.

In particular:

- Perplexity: The perplexity is related to the number of nearest neighbors that is used in other manifold learning algorithms. Larger datasets usually require a larger perplexity. Consider selecting a value between 5 and 50. Different values can result in significantly different results. In general, set to a high value and test a few.
- n_iter: The method must converge to be good. Set high and let the min_grad_norm and n_iter_without_progress stop the training.
- Metric: How to measure distances. Can be any keyword [here](https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.distance.pdist.html). For text, cosine similarity is [known to work](https://academic.oup.com/bioinformatics/article/22/18/2298/318080).

The other parameters are not as significant. Let's train the model!



In [ ]:
TSNE_embedding =

In [ ]:
# Create plot
sns.scatterplot(x=TSNE_embedding[:, 0], y=TSNE_embedding[:, 1], hue=texts['class'])
plt.xlabel("Component 1")
plt.ylabel("Component 2")
plt.title("t-SNE projection of IMDB texts")
plt.show()

We can see that the non-linear projection does help separate some of the classes better than PCA. Play around with the parameters to get a better result!

### UMAP

UMAP (Uniform Manifold Approximation and Projection) is a more sophisticated model based on solid mathematical principles. They are fairly sophisticated, so if you want to check them out in detail, read the [original paper](https://arxiv.org/abs/1802.03426). In particular, Appendix C is of great use if you are familiar with Machine Learning notation.

UMAP is far more efficient than t-SNE, particularly when projecting into more than two dimensions, so it is generally a better method than t-SNE. It is, however, still not very mainstream.

Let's create a 2D projection of our data using UMAP, that works great with sparse matrices.

In [ ]:
# Let's create the object
reducer = umap.UMAP(n_neighbors=15,              # Number of neareast neighbours to use.
                    n_components=2,              # Number of components. UMAP is robust to larger values
                    metric='hellinger',          # Metric to use.
                    n_epochs=None,               # Iterations. Set to convergence. None implies either 200 or 500.
                    min_dist=0.1,                # Minimum distance embedded points. Smaller makes clumps, larger, sparseness.
                    spread=1.0,                  # Scale to combine with min_dist
                    low_memory=True,             # Run slower, but with less memory.
                    n_jobs=-1,                   # Cores to use
                    random_state=seed,           # Random seed
                    verbose=False                # Verbosity
                   )

# Now we train and calculate the embedding!
UMAP_embedding =

In [ ]:
# Create plot
sns.scatterplot(x=UMAP_embedding[:, 0], y=UMAP_embedding[:, 1], hue=texts['class'])
plt.xlabel("Component 1")
plt.ylabel("Component 2")
plt.title("UMAP of IMDB texts")
plt.show()

Or alternatively (and using way less memory) we can simply plot the mapper directly.

In [ ]:
umap.plot.points(reducer, labels=texts['class'])
plt.show()

UMAP and t-SNE create much sparser divisions, and one that clearly separates both classes! What we can infer is that there is a significant non-linear separation between these two movie reviews, and that this can be correctly interpreted using non-linear models.